In [1]:
# Script: Eststimate_MI_IStar_NonLinear_Model.ipynb

In [2]:
# Step 1. Import all Packages: Pandas, Numpy, and Statistics
import pandas as pd
import numpy as np
#import statistics as st
import scipy
#from scipy import stats
import scipy.optimize as spo
from scipy.optimize import minimize
from scipy.optimize import fsolve

In [3]:
# Step 2. User Inputs
import os
current_working_directory = os.getcwd()

InputPath = current_working_directory + "\\"
OutputPath = current_working_directory + "\\"
print("InputPath: {0}, OutputPath: {1}".format(InputPath, OutputPath))

MIDataFile="MI_Data.csv"
MIParameterFile="MI_Parameters.csv"


InputPath: C:\Users\nburg\Desktop\MMT04\, OutputPath: C:\Users\nburg\Desktop\MMT04\


In [4]:
# Step 3. Read MIData file
MI_df = pd.read_csv(InputPath+MIDataFile)
MI_df.head()

,Date,Size,Volatility,POV,Cost
0,8/14/2020,0.0927,0.498829,0.0303,-0.357895
1,8/14/2020,0.0568,0.344376,0.0333,171.617139
2,8/14/2020,0.0534,0.193020,0.0415,-26.072693
3,8/14/2020,0.0580,0.471967,0.0478,260.844290
4,8/14/2020,0.1500,0.337130,0.0538,126.922787


In [5]:
# Step 4. Define Variables from MIData file
Size = MI_df['Size'].to_numpy()
Volatility = MI_df['Volatility'].to_numpy()
POV = MI_df['POV'].to_numpy()
Cost = MI_df['Cost'].to_numpy()


In [6]:
# Step 5. Define function to calculate Non-Linear Objective Function
def optimize1(initial_guess, Size, POV, Volatility, Cost):
    a1, a2, a3, a4, b1 = initial_guess
    return np.sum(np.square((a1 * (Size ** a2) * (Volatility ** a3) * (b1 * (POV ** a4) + (1 - b1))) - Cost))


In [7]:
# Step 6. Initial Guess for Non-Linear Optimization

# a parameters must be greater than zero
# b parameter must be between zero and one

a1 = 1000  # Market Impact Size Multiplier
a2 = 0.5   # Order Size Shape Parameter
a3 = 0.5   # Volatility Shape Parameter
a4 = 0.5   # POV Shape Parameter
b1 = 0.95  # Percent of Temporary Market Impact

initial_guess = [a1, a2, a3, a4, b1]

# Set Upper and Lower Bounds on Constraints
LB = [0.0001, 0.0001, 0.0001, 0.0001, 0.00001] 
UB = [2000, 2.0, 2.0, 2.0, 1]

MyBounds = []
i = 0 
for i in range(0,len(LB)):
    MyBounds.append((LB[i], UB[i]))
    i = i+1

result = minimize(optimize1, initial_guess, bounds = MyBounds, args = (Size, POV, Volatility, Cost))
result

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 4945541211.39081
        x: [ 8.835e+02  3.541e-01  7.556e-01  8.261e-01  9.635e-01]
      nit: 26
      jac: [ 1.907e+02 -3.510e+04 -8.907e+04 -1.355e+05 -2.786e+05]
     nfev: 282
     njev: 47
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>

In [8]:
# Step 7. Save MI Parameters to output file
a1 = result.x.tolist()[0]
a2 = result.x.tolist()[1]
a3 = result.x.tolist()[2]
a4 = result.x.tolist()[3]
b1 = result.x.tolist()[4]

new_a1 = pd.Series(result.x.tolist()[0])
new_a2 = pd.Series(result.x.tolist()[1])
new_a3 = pd.Series(result.x.tolist()[2])
new_a4 = pd.Series(result.x.tolist()[3])
new_b1 = pd.Series(result.x.tolist()[4])

#Optimized_SSE = pd.Series(np.sum(np.square((a1 * (Size ** a2) * (Volatility ** a3) * (b1 * (POV ** a4) + (1 - b1))) - Cost)))
#MiConstraints = pd.concat([new_a1 ,new_a2, new_a3, new_a4, new_b1, Optimized_SSE], axis =1)
#MiConstraints.columns = [['a1', 'a2', 'a3', 'a4', 'b1', 'Optimized_SSE']]

MiParameters = pd.concat([new_a1 ,new_a2, new_a3, new_a4, new_b1], axis =1)
MiParameters.columns = [['a1', 'a2', 'a3', 'a4', 'b1']]
MiParameters = MiParameters.transpose()
MiParameters.columns = ['Value']
MiParameters.index.rename('Parameter', inplace=True)

#MiParameters.to_csv(OutputPath+'MiParameters.csv', index = True, header = True)
MiParameters.to_csv(OutputPath+MIParameterFile, index = True, header = True)

In [9]:
# Step 8. Display MI Parameters to screen
MiParameters

,Value
Parameter,
a1,883.490789
a2,0.354069
a3,0.755646
a4,0.826101
b1,0.963511
